In [1]:
import math
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib
import matplotlib.pyplot as plt
#%matplotlib notebook
%matplotlib inline
import datetime
import tables 
import tstables  
import os
from indicators import *
from sklearn import svm

def write2excel( df, filename ):
    filepath = os.path.join('..', '..', 'datastore', filename) + '.xlsx'
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    df.to_excel(writer )
    writer.save()

In [2]:
symbol = 'AUD_USD'
account_type = 'practice'
granularity = 'S5'
start_datetime = datetime.datetime(2010,1,1,0,0,0)
end_datetime = datetime.datetime(2015,1,1,0,0,0)
decision_frequency = '4H'

file_path = '..\\..\\datastore\\_{0}\\{1}\\{2}.h5'.format(account_type,symbol,granularity)

h5 = tables.open_file(file_path, 'r')
ts = h5.root.data._f_get_timeseries()
raw = ts.read_range(start_datetime,end_datetime)
raw = pd.DataFrame(raw)
                
# Aggregate the high frequency data to the decision frequency
ohlc_dict = {   'ask_o':'first', 'ask_h':'max', 'ask_l':'min', 'ask_c': 'last',                                                                                                    
                'bid_o':'first', 'bid_h':'max', 'bid_l':'min', 'bid_c': 'last',                                                                                                    
                'volume': 'sum' }

df = raw.resample(decision_frequency, closed='left', label='left').apply(ohlc_dict).dropna()

indicator_list = []

In [249]:
df.columns

Index([              u'closeAsk',                  u'Close',
                      u'highAsk',                   u'High',
                       u'lowAsk',                    u'Low',
                      u'openAsk',                   u'Open',
                         u'time',                 u'volume',
                    u'WaveClose',               u'WaveHigh',
                      u'WaveLow',              u'WaveAngle',
                         u'macd',             u'macdsignal',
                     u'macdhist',              u'dmi_minus',
                     u'dmi_plus',               u'dmi_diff',
                          u'RSI',                  u'slowk',
                        u'slowd',                    u'atr',
                        u'atr+1',                  u'atr+2',
                        u'atr+3',                  u'atr-1',
                        u'atr-2',                  u'atr-3',
                    u'MinorHigh',               u'MinorLow',
         u'DivergenceReg

In [250]:
cols = ['Close','High','Low','Open','WaveAngle','macd','macdsignal','macdhist','dmi_plus','dmi_minus','dmi_diff','atr+1','atr-1']
df2 = df[cols]

In [251]:
df2['return'] = df2['Close']/df2['Close'].shift(1)


c:\users\bora\appdata\local\enthought\canopy\user\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [252]:
def f(row):
    if row['return'] >= 1:
        val = 1
    else:
        val = 0
    return val

df2['UpDown'] = df2.apply(f, axis=1)

df2['Y'] =  df2['UpDown'].shift(-1)


c:\users\bora\appdata\local\enthought\canopy\user\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\bora\appdata\local\enthought\canopy\user\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [253]:
cols = []

lookback = 30
for i in range(1,lookback+1):
    columnname = 'C_C' + str(i)    
    df2[columnname] =  df2['Close'] / df2['Close'].shift(i)
    cols.append(columnname)
    
    columnname = 'C_H' + str(i)    
    df2[columnname] =  df2['Close'] / df2['High'].shift(i)
    cols.append(columnname)
    
    columnname = 'C_L' + str(i)    
    df2[columnname] =  df2['Close'] / df2['Low'].shift(i)
    cols.append(columnname)
    
    columnname = 'macd_macd' + str(i)    
    df2[columnname] =  df2['macd'] / df2['macd'].shift(i)
    cols.append(columnname)
    
    columnname = 'macdsignal_macdsignal' + str(i)    
    df2[columnname] =  df2['macdsignal'] / df2['macdsignal'].shift(i)
    cols.append(columnname)
    
    columnname = 'macdhist_macdhist' + str(i)    
    df2[columnname] =  df2['macdhist'] / df2['macdhist'].shift(i)
    cols.append(columnname)

    columnname = 'WaveAngle_WaveAngle' + str(i)    
    df2[columnname] =  df2['WaveAngle'] / df2['WaveAngle'].shift(i)
    cols.append(columnname)

    columnname = 'dmi_diff_dmi_diff' + str(i)    
    df2[columnname] =  df2['dmi_diff'] / df2['dmi_diff'].shift(i)
    cols.append(columnname)
    
    columnname = 'dmi_plus_dmi_plus' + str(i)    
    df2[columnname] =  df2['dmi_plus'] / df2['dmi_plus'].shift(i)
    cols.append(columnname)

    columnname = 'dmi_minus_dmi_minus' + str(i)    
    df2[columnname] =  df2['dmi_minus'] / df2['dmi_minus'].shift(i)
    cols.append(columnname)

    columnname = 'atr+1_atr+1_' + str(i)    
    df2[columnname] =  df2['atr+1'] / df2['atr+1'].shift(i)
    cols.append(columnname)
    
    columnname = 'atr-1_atr-1_' + str(i)    
    df2[columnname] =  df2['atr-1'] / df2['atr-1'].shift(i)
    cols.append(columnname)
    

c:\users\bora\appdata\local\enthought\canopy\user\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\bora\appdata\local\enthought\canopy\user\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\bora\appdata\local\enthought\canopy\user\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [254]:
cols2.extend(cols)

df2=df2.dropna()
    

In [256]:
df2.head()

,Close,High,Low,Open,WaveAngle,macd,macdsignal,macdhist,dmi_plus,dmi_minus,...,C_L30,macd_macd30,macdsignal_macdsignal30,macdhist_macdhist30,WaveAngle_WaveAngle30,dmi_diff_dmi_diff30,dmi_plus_dmi_plus30,dmi_minus_dmi_minus30,atr+1_atr+130,atr-1_atr-130
66,1.30830,1.31909,1.30518,1.31533,-0.989544,-0.002007,0.000797,-0.002804,25.365506,18.814418,...,0.994512,-0.187288,0.070808,5.164426,-0.269443,1.362336,1.153892,1.095527,1.011752,1.015545
67,1.30317,1.30899,1.30108,1.30828,-1.209117,-0.003014,0.000035,-0.003049,23.866683,20.765461,...,0.988913,-0.295411,0.003160,3.612774,-0.377646,0.658225,1.108114,1.234084,1.010604,1.013582
68,1.30793,1.31189,1.30037,1.30314,-1.412772,-0.003389,-0.000650,-0.002739,23.977531,19.004225,...,0.991938,-0.355600,-0.060492,2.258414,-0.509301,0.543762,0.962272,1.204971,1.009644,1.012208
69,1.31705,1.31865,1.30486,1.30795,-1.216652,-0.002916,-0.001103,-0.001813,26.445863,17.131135,...,1.006950,-0.344127,-0.107217,0.999583,-0.527195,5.599845,1.161095,0.811392,1.009278,1.011644
70,1.31705,1.31705,1.31705,1.31705,-0.598706,-0.002513,-0.001385,-0.001128,26.445863,17.131135,...,1.009752,-0.358701,-0.143801,0.429529,-0.343158,-12.703425,1.276907,0.798873,1.008424,1.012243


In [258]:
X = df2[cols2].values
y = df2['Y'].values

In [225]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
cols.extend(['WaveAngle','macd','macdsignal','macdhist','dmi_plus','dmi_minus','dmi_diff','atr+1','atr-1'])

X = df2[cols].values
y = df2['Y'].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# lr = LogisticRegression()
# lr.fit(X_train, y_train)
# lr.score(X_train, y_train)

# probs = lr.predict_proba(X_test)
# predicted = lr.predict(X_test)
# lr.score(X_test, y_test)

In [226]:
df2.head(50)

,Close,High,Low,Open,WaveAngle,macd,macdsignal,macdhist,dmi_plus,dmi_minus,...,C_C30,C_H30,C_L30,macd_macd30,macdsignal_macdsignal30,macdhist_macdhist30,WaveAngle_WaveAngle30,dmi_diff_dmi_diff30,dmi_plus_dmi_plus30,dmi_minus_dmi_minus30
66,1.30830,1.31909,1.30518,1.31533,-0.989544,-0.002007,0.000797,-0.002804,25.365506,18.814418,...,0.991730,0.984395,0.994512,-0.187288,0.070808,5.164426,-0.269443,1.362336,1.153892,1.095527
67,1.30317,1.30899,1.30108,1.30828,-1.209117,-0.003014,0.000035,-0.003049,23.866683,20.765461,...,0.986876,0.986533,0.988913,-0.295411,0.003160,3.612774,-0.377646,0.658225,1.108114,1.234084
68,1.30793,1.31189,1.30037,1.30314,-1.412772,-0.003389,-0.000650,-0.002739,23.977531,19.004225,...,0.991938,0.984642,0.991938,-0.355600,-0.060492,2.258414,-0.509301,0.543762,0.962272,1.204971
69,1.31705,1.31865,1.30486,1.30795,-1.216652,-0.002916,-0.001103,-0.001813,26.445863,17.131135,...,1.002855,0.996587,1.006950,-0.344127,-0.107217,0.999583,-0.527195,5.599845,1.161095,0.811392
70,1.31705,1.31705,1.31705,1.31705,-0.598706,-0.002513,-0.001385,-0.001128,26.445863,17.131135,...,1.008106,0.998529,1.009752,-0.358701,-0.143801,0.429529,-0.343158,-12.703425,1.276907,0.798873
71,1.31732,1.31755,1.31601,1.31699,-0.120532,-0.002147,-0.001538,-0.000609,26.112526,17.766418,...,1.003313,1.001110,1.015424,-0.341060,-0.171504,0.228286,-0.112621,-1.733261,1.416637,0.764212
72,1.32376,1.32651,1.31416,1.31736,0.231607,-0.001322,-0.001494,0.000172,30.671375,16.022280,...,1.007842,1.003061,1.009363,-0.231704,-0.179767,-0.066136,0.223674,-6.560777,1.544921,0.725454
73,1.32239,1.32514,1.31721,1.32379,0.414840,-0.000770,-0.001349,0.000580,28.721617,15.003755,...,1.006416,1.006416,1.006646,-0.147515,-0.175392,-0.234142,0.361315,-6.163782,1.451437,0.681556
74,1.32150,1.32844,1.31785,1.32237,0.635506,-0.000399,-0.001159,0.000760,28.925991,13.746968,...,1.002800,1.001098,1.006497,-0.078548,-0.161663,-0.364029,0.440655,7.789770,1.261974,0.655472
75,1.31997,1.32538,1.31334,1.32148,0.516166,-0.000227,-0.000973,0.000746,26.235304,15.952605,...,0.996806,0.994245,1.002171,-0.041731,-0.142573,-0.535704,0.301424,1.496935,0.994944,0.818105


In [259]:
lr = LogisticRegression()
lr.fit(X, y)
lr.score(X, y)

probs = lr.predict_proba(X)
predicted = lr.predict(X)
lr.score(X, y)


0.6417352281226627

In [260]:
df2['predicted'] = predicted

In [261]:
df2['probs0'] = probs[:,0]
df2['probs1'] = probs[:,1]

In [264]:
df2[['Open','Close','Y','predicted','probs0','probs1']].tail(50)

,Open,Close,Y,predicted,probs0,probs1
1353,1.10533,1.10963,1.0,0.0,0.778438,0.221562
1354,1.10964,1.11049,1.0,0.0,0.558573,0.441427
1355,1.11033,1.11352,0.0,0.0,0.643920,0.356080
1356,1.10556,1.10394,0.0,0.0,0.630254,0.369746
1357,1.10397,1.10236,0.0,0.0,0.519519,0.480481
1358,1.10245,1.09074,0.0,1.0,0.498957,0.501043
1359,1.09074,1.08888,0.0,0.0,0.870132,0.129868
1360,1.08914,1.08516,0.0,0.0,0.690843,0.309157
1361,1.08304,1.07356,0.0,0.0,0.873560,0.126440
1362,1.07359,1.07211,0.0,0.0,0.635187,0.364813


In [265]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
clf = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y)
scores.mean()   


0.51236290959170994

In [266]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=10)
scores = cross_val_score(clf, X, y)
scores.mean()       

0.5198300330864446

In [267]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=10, learning_rate=10,max_depth=1, random_state=0).fit(X, y)
scores = cross_val_score(clf, X, y)
scores.mean()   

0.50486219579785363

In [268]:
fname = os.path.join('all.xlsx')
writer = pd.ExcelWriter(fname, engine='xlsxwriter')
df2.to_excel(writer )
writer.save()